## Plot Ideas
-------------

* QQ Normal Plot to see if Hailstone Sizes are normally distributed
    * If not normal, then find a distribution that does fit it (maybe something log normal?)
    * Generate plot that demonstrates this

* Histograms for some of the variables, especially Hailstone Sizes and maybe heatmaps with some other ones?

* For all duplicates, see how far apart the actual variables are; worth using three times as much information for little benefit?
    * Do this maybe with... stacked histograms / line plot / something else? 
    * Calculate mean of duplicate variables, is that a better indicator, or should use closest to mean variable?

* Correlation matrix for the data, make it real pretty like, consider whether we neeeeed all these variables or can PCA/SVM/LASSO to reduce dimensionality

* Scale data maybe? 

* Boxplots to see about spread and central tendency, maybe even two dimensional versions or facet grid

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from math import floor
import statsmodels.api as sm


In [15]:
%matplotlib qt
rng = np.random.default_rng(100)
font = {'family' : 'Comic Sans MS',
        'weight' : 'bold',
        'size'   : "40"}


In [3]:
path = "/Users/joshuaelms/Desktop/github_repos/CSCI-B365/Meteorology_Modeling_Project/data/pretty_data.csv"

df = pd.read_csv(path, index_col=0)
df.index += 1
df.iloc[:, [0, 10, 20]].corr()

,ML CAPE,MU CAPE,SB CAPE
ML CAPE,1.000000,0.926460,0.879841
MU CAPE,0.926460,1.000000,0.943481
SB CAPE,0.879841,0.943481,1.000000


In [16]:
plt.clf()
# plt.rcParams["font.family"] = "Party LET"
plt.rc("font", **font)

x = [1, 2, 3, 4]
y = [10, 8, 6, 4]

fig, [ax1, ax2] = plt.subplots(2, 1)

sns.set_theme()
plt.suptitle("The Quick Brown Fox Jumped Over the Lazy Dog", family="Marker Felt")
sns.pointplot(x=y, y=x, ax=ax1)
sns.pointplot(x=x, y=y, ax=ax2)
ax1.set_xlabel("Testing")
plt.show()

In [77]:
### Histogram and QQ Norm for Hail Size

step = 0.25
breaks = [i for i in np.arange(floor(df["Hailstone Size"].min()), df["Hailstone Size"].max() + step, step)]

normal = rng.standard_normal(size=df["Hailstone Size"].shape[0])

fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, sharex=False, sharey=False)
fig.patch.set_facecolor("xkcd:powder blue")
sns.histplot(data=df, x="Hailstone Size", discrete=True, bins=breaks, ax=ax1)
sm.qqplot(data=df["Hailstone Size"], line="45", ax=ax2)
ax1.legend(["Flirst"])
ax2.legend(["Stuff", "More"])
ax1.set_title("First")
ax2.set_title("Second")
plt.tight_layout()
plt.show()

/var/folders/95/7xkz0c3d73z_7dvzl3nvx1g80000gn/T/ipykernel_44859/2861048494.py:16: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  plt.tight_layout()
/Users/joshuaelms/.virtualenvs/rising_sun/lib/python3.10/site-packages/ipykernel/eventloops.py:105: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  app.exec_()


In [24]:
### Corr plot for ten duplicates
plt.clf()

### group plots by variable; for each variable in the dictionary, generate and display corrplot of various calculation methods for it 
fig, ax_lst = plt.subplots(nrows=5, ncols=2, figsize=(8, 10))

cnt = 0
for layer in ax_lst:
    for ax in layer:
        correlations = df.iloc[:, [cnt, cnt+10, cnt+20]].corr()
        title = correlations.columns[0].split()[-1]
        sns.heatmap(data=correlations, vmin=-1, vmax=1, ax=ax, cmap="magma")
        ax.tick_params(axis='x', rotation=0)
        ax.set_title(title)
        cnt+=1
        
fig.suptitle("Correlation Plots for 3 Methods of Calculating 10 Meteorological Metrics")
fig.patch.set_facecolor("xkcd:light grey")
plt.tight_layout()
plt.show()


In [55]:
### Corr plot overall

plt.clf()

desired = [i for i in range(20,46)] + [i for i in range(48, 54)]

fig, ax1 = plt.subplots()
df_corr = df.corr().iloc[desired, desired]

sns.heatmap(data=df_corr, vmin=-1, vmax=1, ax=ax1,  xticklabels=1, yticklabels=1)

plt.tight_layout()